In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets
import torchvision.utils as vutils
import random
import os
from tensorboardX import SummaryWriter
torch.manual_seed(1)

DATA_PATH = os.environ['DATA_PATH']
USE_CUDA = torch.cuda.is_available()

import matplotlib.pyplot as plt
%matplotlib inline

https://www.cs.unc.edu/~wliu/papers/GoogLeNet.pdf

### Load Dataset

In [2]:
BATCH_SIZE=32

## Residual Module 

In [20]:
class Residual(nn.Module):
    def __init__(self,in_channel,out_channel=None):
        super(Residual,self).__init__()
        
        if out_channel:
            self.increase_depth = True
        else:
            self.increase_depth = False
            out_channel = in_channel
        
        self.residual_layer = nn.Sequential(
                                                    nn.Conv2d(in_channel,out_channel,3,1,1),
                                                    nn.ReLU(),
                                                    nn.BatchNorm2d(out_channel),
                                                    nn.Conv2d(out_channel,out_channel,3,1,1),
                                                    nn.ReLU(),
                                                    nn.BatchNorm2d(out_channel),
                                                )
        if self.increase_depth:
            self.conv1_layer = nn.Sequential(
                                                            nn.Conv2d(in_channel,out_channel,1),
                                                            nn.BatchNorm2d(out_channel)
                                                            )
            
    def forward(self,inputs):
        residual = self.residual_layer(inputs)
        if self.increase_depth:
            inputs = self.conv1_layer(inputs)
        return F.relu(residual+inputs)

In [25]:
writer = SummaryWriter(comment='-basic-residual')
sample_images = torch.randn(32,3,32,32)
residual = Residual(3,6)
output = residual(Variable(sample_images))
writer.add_graph(residual,output)
writer.close()

In [26]:
class BottleNeckResidual(nn.Module):
    def __init__(self,in_channel,out_channel=None):
        super(BottleNeckResidual,self).__init__()
        
        if out_channel:
            self.increase_depth = True
        else:
            self.increase_depth = False
            out_channel = in_channel
        
        self.residual_layer = nn.Sequential(
                                                    nn.Conv2d(in_channel,out_channel,3,1,1),
                                                    nn.ReLU(),
                                                    nn.BatchNorm2d(out_channel),
                                                    nn.Conv2d(out_channel,out_channel,3,1,1),
                                                    nn.ReLU(),
                                                    nn.BatchNorm2d(out_channel),
                                                )
        if self.increase_depth:
            self.conv1_layer = nn.Sequential(
                                                            nn.Conv2d(in_channel,out_channel,1),
                                                            nn.BatchNorm2d(out_channel)
                                                            )
            
    def forward(self,inputs):
        residual = self.residual_layer(inputs)
        if self.increase_depth:
            inputs = self.conv1_layer(inputs)
        return F.relu(residual+inputs)

## Sanity Check 

In [6]:
model = GoogLeNet(1000)

In [7]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

writer = SummaryWriter(comment="-googlenet")

In [ ]:
%time
model.train()
for batch in train_loader:
    x,_ = batch
    outputs, outputs2 = model(Variable(x),True)
    dummy_y = torch.randperm(BATCH_SIZE)
    
    writer.add_graph(model,outputs)
    loss_1 = loss_function(outputs,Variable(dummy_y))
    loss_2 = loss_function(outputs2,Variable(dummy_y))

    loss = loss_1 + 0.3*loss_2
    loss.backward()
    optim.step()
    break

writer.close()

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.63 µs
